# Proof of Concept: Approximate Mean

In [34]:
range_list = [
            dict(min=2499, max=9999, n=7942251, moe=17662),
            dict(min=10000, max=14999, n=5768114, moe=16409),
            dict(min=15000, max=19999, n=5727180, moe=16801),
            dict(min=20000, max=24999, n=5910725, moe=17864),
            dict(min=25000, max=29999, n=5619002, moe=16113),
            dict(min=30000, max=34999, n=5711286, moe=15891),
            dict(min=35000, max=39999, n=5332778, moe=16488),
            dict(min=40000, max=44999, n=5354520, moe=15415),
            dict(min=45000, max=49999, n=4725195, moe=16890),
            dict(min=50000, max=59999, n=9181800, moe=20965),
            dict(min=60000, max=74999, n=11818514, moe=30723),
            dict(min=75000, max=99999, n=14636046, moe=49159),
            dict(min=100000, max=124999, n=10273788, moe=47842),
            dict(min=125000, max=149999, n=6428069, moe=37952),
            dict(min=150000, max=199999, n=6931136, moe=37236),
            dict(min=200000, max=250001, n=7465517, moe=42206)
        ]


Example from table B19001 found [here](https://www2.census.gov/programs-surveys/acs/replicate_estimates/2017/data/5-year/010/)

HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2017 INFLATION-ADJUSTED DOLLARS)

## Generate Own Replicates

In [ ]:
import numpy

range_list.sort(key=lambda x: x['min'])


simR = []        
for i in range(50): ## simulation
    test = []
    track_n = []
    for range_ in range_list:
        se = range_['moe']/1.645 ## convert moe to se
        nn = round(numpy.random.normal(range_['n'], se))
        ## use moe to introduce randomness into number in bin
        nn=int(nn) ## clean it up
        test.append(numpy.random.uniform(range_['min'], range_['max'], size=(1, nn )).sum())
        ## draw random values within the bin, assume uniform
        track_n.append(nn)
    simR.append(sum(test)/sum(track_n)) ## get mean in one simulation iteration 
## why so slow?    

In [ ]:
numpy.mean(simR) ## mean of means


In [ ]:
numpy.quantile(simR,0.05) ## uncertainty lower bound 90%


In [ ]:
numpy.quantile(simR,0.95) ## Uncertainty upper bound 90%

In [ ]:
se=max(numpy.quantile(simR,0.95) -numpy.mean(simR),numpy.mean(simR)-numpy.quantile(simR,0.05) )
se

In [ ]:
moe=1.645*se
moe

Compare to median:

In [20]:
def approximate_median(range_list, design_factor=None, sampling_percentage=None):
    """
    Estimate a median and approximate the margin of error.
    Follows the U.S. Census Bureau's `official guidelines`_ for estimation using a design factor.
    Useful for generating medians for measures like household income and age when aggregating census geographies.
    Args:
        range_list (list): A list of dictionaries that divide the full range of data values into continuous categories.
            Each dictionary should have three keys:
                * min (int): The minimum value of the range
                * max (int): The maximum value of the range
                * n (int): The number of people, households or other unit in the range
            The minimum value in the first range and the maximum value in the last range can be tailored to the dataset
            by using the "jam values" provided in the `American Community Survey's technical documentation`_.
        design_factor (float, optional): A statistical input used to tailor the standard error to the
            variance of the dataset. The Census Bureau publishes design factors as part of its PUMS Accuracy statement.
            Find the value for the dataset you are estimating by referring to `the bureau's reference material`_.
            If you do not provide this input, a margin of error will not be returned.
        sampling_percentage (float, optional): A statistical input used to correct variance for finite population.
            For example, the 1-year ACS is designed to be a 2.5% sample of the population, and the 1-year PUMS is
            designed to be a 1% sample of the population. You can multiply these percentages by 5 for the 5-year versions.
    Returns:
        A two-item tuple with the median followed by the approximated margin of error.
        (42211.096153846156, 10153.200960954948)
    Examples:
        Estimating the median for a range of median household incomes.
        >>> income = [
            dict(min=2499, max=9999, n=186),
            dict(min=10000, max=14999, n=78),
            dict(min=15000, max=19999, n=98),
            dict(min=20000, max=24999, n=287),
            dict(min=25000, max=29999, n=142),
            dict(min=30000, max=34999, n=90),
            dict(min=35000, max=39999, n=107),
            dict(min=40000, max=44999, n=104),
            dict(min=45000, max=49999, n=178),
            dict(min=50000, max=59999, n=106),
            dict(min=60000, max=74999, n=177),
            dict(min=75000, max=99999, n=262),
            dict(min=100000, max=124999, n=77),
            dict(min=125000, max=149999, n=100),
            dict(min=150000, max=199999, n=58),
            dict(min=200000, max=250001, n=18)
        ]
        >>> approximate_median(income, design_factor=1.5, sampling_percentage=1)
        (42211.096153846156, 10153.200960954948)
    ... _official guidelines:
        https://www.documentcloud.org/documents/6165603-2013-2017AccuracyPUMS.html#document/p18
    ... _American Community Survey's technical documentation
        https://www.documentcloud.org/documents/6165752-2017-SummaryFile-Tech-Doc.html#document/p20/a508561
    ... _the bureau's reference material:
        https://www.census.gov/programs-surveys/acs/technical-documentation/pums/documentation.html
    """
    # Sort the list
    range_list.sort(key=lambda x: x['min'])

    # For each range calculate its min and max value along the universe's scale
    cumulative_n = 0
    for range_ in range_list:
        range_['n_min'] = cumulative_n
        cumulative_n += range_['n']
        range_['n_max'] = cumulative_n

    # What is the total number of observations in the universe?
    n = sum([d['n'] for d in range_list])

    # What is the estimated midpoint of the n?
    n_midpoint = n / 2.0

    # Now use those to determine which group contains the midpoint.
    n_midpoint_range = next(d for d in range_list if n_midpoint >= d['n_min'] and n_midpoint <= d['n_max'])

    # How many households in the midrange are needed to reach the midpoint?
    n_midrange_gap = n_midpoint - n_midpoint_range['n_min']

    # What is the proportion of the group that would be needed to get the midpoint?
    n_midrange_gap_percent = n_midrange_gap / n_midpoint_range['n']

    # Apply this proportion to the width of the midrange
    n_midrange_gap_adjusted = (n_midpoint_range['max'] - n_midpoint_range['min']) * n_midrange_gap_percent

    # Estimate the median
    estimated_median = n_midpoint_range['min'] + n_midrange_gap_adjusted

    # If there's no design factor, we can't calculate a margin of error
    if not design_factor:
        # Let's throw a warning, but still return the median
        warnings.warn("", DesignFactorWarning)
        return estimated_median, None

    # If there's no sampling percentage, we can't calculate a margin of error
    if not sampling_percentage:
        # Let's throw a warning, but still return the median
        warnings.warn("", SamplingPercentageWarning)
        return estimated_median, None

    # Get the standard error for this dataset
    standard_error = (design_factor * math.sqrt(((100 - sampling_percentage) / (n * sampling_percentage)) * (50**2))) / 100

    # Use the standard error to calculate the p values
    p_lower = (.5 - standard_error)
    p_upper = (.5 + standard_error)

    # Estimate the p_lower and p_upper n values
    p_lower_n = n * p_lower
    p_upper_n = n * p_upper

    # Find the ranges the p values fall within
    try:
        p_lower_range_i, p_lower_range = next(
            (i, d) for i, d in enumerate(range_list)
            if p_lower_n >= d['n_min'] and p_lower_n <= d['n_max']
        )
    except StopIteration:
        raise DataError(f"The n's lower p value {p_lower_n} does not fall within a data range.")

    try:
        p_upper_range_i, p_upper_range = next(
            (i, d) for i, d in enumerate(range_list)
            if p_upper_n >= d['n_min'] and p_upper_n <= d['n_max']
        )
    except StopIteration:
        raise DataError(f"The n's upper p value {p_upper_n} does not fall within a data range.")

    # Use these values to estimate the lower bound of the confidence interval
    p_lower_a1 = p_lower_range['min']
    try:
        p_lower_a2 = range_list[p_lower_range_i + 1]['min']
    except IndexError:
        p_lower_a2 = p_lower_range['max']
    p_lower_c1 = p_lower_range['n_min'] / n
    try:
        p_lower_c2 = range_list[p_lower_range_i + 1]['n_min'] / n
    except IndexError:
        p_lower_c2 = p_lower_range['n_max'] / n
    lower_bound = ((p_lower - p_lower_c1) / (p_lower_c2 - p_lower_c1)) * (p_lower_a2 - p_lower_a1) + p_lower_a1

    # Same for the upper bound
    p_upper_a1 = p_upper_range['min']
    try:
        p_upper_a2 = range_list[p_upper_range_i + 1]['min']
    except IndexError:
        p_upper_a2 = p_upper_range['max']
    p_upper_c1 = p_upper_range['n_min'] / n
    try:
        p_upper_c2 = range_list[p_upper_range_i + 1]['n_min'] / n
    except IndexError:
        p_upper_c2 = p_upper_range['n_max'] / n
    upper_bound = ((p_upper - p_upper_c1) / (p_upper_c2 - p_upper_c1)) * (p_upper_a2 - p_upper_a1) + p_upper_a1

    # Calculate the standard error of the median
    standard_error_median = 0.5 * (upper_bound - lower_bound)

    # Calculate the margin of error at the 90% confidence level
    margin_of_error = 1.645 * standard_error_median

    # Return the result
    return estimated_median, margin_of_error


In [ ]:
import math
approximate_median(range_list, design_factor=1.5, sampling_percentage=1)

Make replicable with seed

In [36]:
def approximate_mean(range_list, number_replicates=50, rng=None):
    """
    Estimate a mean and approximate the margin of error.

    

    Args:
        range_list (list): A list of dictionaries that divide the full range of data values into continuous categories.
            Each dictionary should have three keys:
                * min (int): The minimum value of the range
                * max (int): The maximum value of the range
                * n (int): The number of people, households or other unit in the range
            The minimum value in the first range and the maximum value in the last range can be tailored to the dataset
            by using the "jam values" provided in the `American Community Survey's technical documentation`_.
        number_replicates (int): number of replicates for simulation, used to estimate margin of error
        rng (int, optional): a seed, used to ensure replicability due to randomness in simulation

    Returns:
        A two-item tuple with the mean followed by the approximated margin of error.

        (42211.096153846156, 10153.200960954948)

    Examples:
        Estimating the median for a range of median household incomes.

        >>> income = [
            dict(min=2499, max=9999, n=186),
            dict(min=10000, max=14999, n=78),
            dict(min=15000, max=19999, n=98),
            dict(min=20000, max=24999, n=287),
            dict(min=25000, max=29999, n=142),
            dict(min=30000, max=34999, n=90),
            dict(min=35000, max=39999, n=107),
            dict(min=40000, max=44999, n=104),
            dict(min=45000, max=49999, n=178),
            dict(min=50000, max=59999, n=106),
            dict(min=60000, max=74999, n=177),
            dict(min=75000, max=99999, n=262),
            dict(min=100000, max=124999, n=77),
            dict(min=125000, max=149999, n=100),
            dict(min=150000, max=199999, n=58),
            dict(min=200000, max=250001, n=18)
        ]
        >>> approximate_mean(income)
        (42211.096153846156, 10153.200960954948)
    """
    
    if rng is not None:
        numpy.random.seed(rng)
            
    # Sort the list
    range_list.sort(key=lambda x: x['min'])

    simR = []        
    for i in range(number_replicates): ## simulation
        test = []
        track_n = []
        for range_ in range_list:
            se = range_['moe'] / 1.645 ## convert moe to se
            nn = round(numpy.random.normal(range_['n'], se))
            ## use moe to introduce randomness into number in bin
            nn = int(nn) ## clean it up
            test.append(numpy.random.uniform(range_['min'], range_['max'], size=(1, nn)).sum())
            ## draw random values within the bin, assume uniform
            track_n.append(nn)
        simR.append(sum(test) / sum(track_n))
    
    estimated_mean = numpy.mean(simR)
    se = max(numpy.quantile(simR, 0.95) - numpy.mean(simR), numpy.mean(simR) - numpy.quantile(simR, 0.05))
    margin_of_error = 1.645 * se
    
    # Return the result
    return estimated_mean, margin_of_error

In [37]:
estimate, moe = approximate_mean(range_list, rng=711355) ## used for testing purposes


In [ ]:
estimate

moe

In [ ]:
estimate, moe = approximate_mean(range_list)

In [ ]:
estimate

In [ ]:
moe